In [30]:
import simpy as sim
import numpy as np
import math
import pandas as pd
# Suppress SettingWithCopyWarning
pd.options.mode.chained_assignment = None

import random
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from collections import defaultdict
from collections import namedtuple
import itertools
import os
import pickle
import time
import optimization_module as om
from gurobipy import Model, GRB

In [31]:
# Simulation Settings -----------------------------------------------------------
number_of_simulation = 1
simulation_duration = 6*7*1440
random_seed = True
random_seed_value = 0 #Only applies if random_seed is False
print_event_enabled = True
print_plot = False
print_output = True
apply_disruption = True
start_from_0 = True
print('Simulation Settings:')
print('Number of simulation:', number_of_simulation)
print('Random seed:', random_seed)
print('Random seed value:', random_seed_value)

Simulation Settings:
Number of simulation: 1
Random seed: True
Random seed value: 0


In [32]:
#Load the last output of the training
# with open('total_cost_200.pkl', 'rb') as f:
#     total_cost_plot_read = pickle.load(f)

# with open('total_reward_200.pkl', 'rb') as f:
#     total_reward_plot_read = pickle.load(f)

policy_name = 'gp'
sd = '3' #disruption set
q_table_path = 'q_table\q_table_200_50000_eps_v2'
experiment_path = f'pkl_output\Experiment1\{policy_name}_{sd}.pkl'
output_path = f'csv_output\Integrated_Model\Experiment 2 Output\V3\{policy_name}_{sd}_{number_of_simulation}.csv'
# len(total_cost_plot_read)
print('Policy:', policy_name)
print('Disruption set:', sd)
print('Q-table path:', q_table_path)
print('Experiment path:', experiment_path)
print('Output path:', output_path)

Policy: gp
Disruption set: 3
Q-table path: q_table\q_table_200_50000_eps_v2
Experiment path: pkl_output\Experiment1\gp_3.pkl
Output path: csv_output\Integrated_Model\Experiment 2 Output\V3\gp_3_1.csv


In [33]:
# ------------------ Read the datasets ------------------ #
path = r"C:\Users\Satrya Dewantara\OneDrive - Delft University of Technology\QX Thesis\Datasets"

def split_to_sublists(input_string):
    sub_lists = input_string.split(";")
    sub_lists = [sub_list.strip().split(", ") for sub_list in sub_lists]
    return sub_lists

def get_first_sublist(sub_lists):
    return sub_lists[0]

def remove_first_sublist(sub_lists):
    return sub_lists[1:]

request = pd.read_csv(f"{path}\\Integrated Model\simulation_input_200_3w_v2.csv")

request.rename(columns={'Demand_ID': 'ID', 'Origin': 'p', 'Destination': 'd', 'Release time': 'ap', 'Due Time': 'bp', 'Container Volume': 'qr', 'k': 'k', 'Actual Announce Time': 'act'}, inplace=True)

# Get the k
request['Solution_List'] = request['Solution_List'].apply(lambda s: [] if s == '0' else split_to_sublists(s))
request['k'] = request['Solution_List'].apply(lambda s: [] if s == [] else get_first_sublist(s))
request['Solution_List'] = request['Solution_List'].apply(lambda s: [] if s == [] else remove_first_sublist(s))

# request = request[['ID', 'p', 'd', 'ap', 'bp','qr', 'k', 'act']]
request = request[['ID', 'p', 'd', 'ap', 'bp','qr', 'k', 'Solution_List', 'act']]
# request['k'] = request['k'].apply(lambda s: [] if s == '0' else s.split(', '))

df_fixed_schedule = pd.read_csv(f"{path}\\Integrated Model\Fixed Vehicle Schedule2.csv")
df_fixed_schedule = df_fixed_schedule.drop(columns=['Travel Cost'])
df_truck_schedule = pd.read_csv(f"{path}\\Integrated Model\Truck Schedule.csv") # Added for truck schedule
df_truck_schedule = df_truck_schedule.drop(columns=['Travel Cost'])
# For optimization module
possible_paths_ref = pd.read_csv(f"{path}\\Integrated Model\\new_paths.csv")

network = pd.read_csv(f"{path}\\Network.csv")
network_barge = pd.read_csv(f"{path}\\Network_Barge.csv")
network_train = pd.read_csv(f"{path}\\Network_Train.csv")
network_truck = pd.read_csv(f"{path}\\Network_Truck.csv")

if apply_disruption:
    s_disruption_profile = pd.read_csv(f"{path}\\Integrated Model\Experiment\Service_Disruption_Profile_{sd}.csv")
    d_disruption_profile = pd.read_csv(f"{path}\\No_Request_Disruption_Profile.csv")
else:
    s_disruption_profile = pd.read_csv(f"{path}\\No_Service_Disruption_Profile.csv")
    d_disruption_profile = pd.read_csv(f"{path}\\No_Request_Disruption_Profile.csv")
combinations = pd.read_csv(f"{path}\\Simple_Network\Possible Combinations.csv")

# Costs
barge_travel_cost1 = 0.6122  # EUR/TEU/hour
barge_travel_cost2 = 0.0213  # EUR/TEU/km
barge_handling_cost = 3  # EUR/TEU

train_travel_cost1 = 7.54  # EUR/TEU/hour
train_travel_cost2 = 0.0635  # EUR/TEU/km
train_handling_cost = 18  # EUR/TEU

truck_travel_cost1 = 30.98  # EUR/TEU/hour
truck_travel_cost2 = 0.2758  # EUR/TEU/km
truck_handling_cost = 18  # EUR/TEU

storage_cost = 1 # EUR/TEU/hour
delay_penalty = 1 # EUR/TEU/hour

penalty_per_unfulfilled_demand = 150000 # For optimization module

undelivered_penalty = 100 # For RL
# Time Param
handling_time = 1 # minute

# ------------------ Data Preprocessing ------------------ #
df_fixed_schedule[['K', 'o', 'o2']] = df_fixed_schedule[['K', 'o', 'o2']].astype("string")
df_truck_schedule[['K', 'o', 'o2']] = df_truck_schedule[['K', 'o', 'o2']].astype("string") # Added for truck schedule

network_ref = network.set_index(['N'])
network_barge_ref = network_barge.set_index(['N'])
network_train_ref = network_train.set_index(['N'])
network_truck_ref = network_truck.set_index(['N'])
s_disruption_profile = s_disruption_profile
d_disruption_profile = d_disruption_profile
d_list = s_disruption_profile['Profile'].tolist() + ['Profile6']

# ------------------ Data Cleaning and Filtering ------------------ #

# Modifying network dataset
network_dict = {i+1: terminal for i, terminal in enumerate(network["N"])}
reverse_dict = {terminal: id for id, terminal in network_dict.items()}
node_list = network["N"].tolist()
train_list = network_train["N"].tolist()
barge_list = network_barge["N"].tolist()

#TEST
# network_full = pd.read_csv(f"{path}\\Network.csv")
# network_dict_full = {i+1: terminal for i, terminal in enumerate(network_full["N"])}

# Add a new column 'Distance' to the fixed_vehicle_schedule
def get_distance(row):
    if 'Barge' in row['K']:
        return network_barge_ref.at[row['o'], row['o2']]
    elif 'Train' in row['K']:
        return network_train_ref.at[row['o'], row['o2']]
    return network_truck_ref.at[row['o'], row['o2']]
df_fixed_schedule['Distance'] = df_fixed_schedule.apply(get_distance, axis=1)
df_truck_schedule['Distance'] = df_truck_schedule.apply(get_distance, axis=1) # Added for truck schedule

# Modifiy the mode schedule for simulation
fixed_list = df_fixed_schedule['K'].unique().tolist()
truck_list = df_truck_schedule['K'].unique().tolist()

mode_list = fixed_list + truck_list
mode_ID = {mode_list[i]: i + 1 for i in range(len(mode_list))}

fixed_schedule = df_fixed_schedule.values.tolist()
for i in range(len(fixed_schedule)):
    if 'Barge' in fixed_schedule[i][0]:
        fixed_schedule[i].append((barge_travel_cost1, barge_travel_cost2, barge_handling_cost))
    else:
        fixed_schedule[i].append((train_travel_cost1, train_travel_cost2, train_handling_cost))

fixed_schedule = [[list[0]] + [(list[1], list[2], list[3]*60)] + list[6:] for list in fixed_schedule] #Create tuple for schedule
fixed_schedule_dict = {fixed_schedule[i][0]: (fixed_schedule[i][0:]) for i in range(len(fixed_schedule))}
# mode_schedule_dict = {fixed_schedule[i][0]: (fixed_schedule[i][0:]) for i in range(len(fixed_schedule))}

# truck_schedule = df_truck_schedule.values.tolist()
truck_schedule = df_truck_schedule.values.tolist()
truck_cost = (truck_travel_cost1, truck_travel_cost2, truck_handling_cost)
for i in range(len(truck_schedule)):
    truck_schedule[i].append(truck_cost)
truck_schedule = [[list[0]] + [(list[1], list[2], list[3]*60)] + list[6:] for list in truck_schedule] #Create tuple for schedule
truck_schedule_dict = {truck_schedule[i][0]: (truck_schedule[i][0:]) for i in range(len(truck_schedule))}

# Modifiy the request dataset for simulation
request.p = request.p.map(network_dict) #Convert terminal ID to terminal name
request.d = request.d.map(network_dict)
request_ids = request['ID'].tolist()
request_list = request.values.tolist()
# request_list = [list[:6] + [[]] + [list[7]] for list in request_list] #Create tuple for request

# Modifiy the disruption profile for simulation
s_disruption_profile = s_disruption_profile[['Profile', 'Impact Type', 'LB Duration', 'UB Duration', 'LB Capacity', 'UB Capacity', 'Location', 'Lambda']]
s_disruption_profile['Location'] = s_disruption_profile['Location'].apply(lambda s: s.split(', '))
s_disruption_profile = s_disruption_profile.values.tolist()
d_disruption_profile = d_disruption_profile[['Profile', 'Impact Type', 'LB Time', 'UB Time', 'LB Volume', 'UB Volume', 'Lambda']]
d_disruption_profile = d_disruption_profile.values.tolist()

# Modifiy the possible combinations for simulation
# combinations['Services'] = combinations['Services'].apply(lambda s: s.split(', '))
# combinations['K'] = combinations['Services'].apply(lambda s: s[0])
# combinations = pd.merge(combinations, fixed_schedule[['K', 'Departure']], on='K', how='left')
# combinations = combinations.sort_values(by='Departure')
# combinations['Departure'] = combinations['Departure'].apply(lambda s: s*60)
# combinations = combinations.values.tolist()

# Modifiy the possible paths for simulation
def map_numbers(number_str):
    numbers = [int(num.strip()) for num in number_str.split(',')]
    if numbers[0] == 0:
        return '0'
    else:
        mapped_numbers = [network_dict[num] for num in numbers]
    return ', '.join(map(str, mapped_numbers))

possible_paths_ref['origin'] = possible_paths_ref['origin'].apply(lambda s: network_dict[s])
possible_paths_ref['destination'] = possible_paths_ref['destination'].apply(lambda s: network_dict[s])
possible_paths_ref['Transshipment Terminal(s)'].fillna('0', inplace=True)
possible_paths_ref['Transshipment Terminal(s)'] = possible_paths_ref['Transshipment Terminal(s)'].astype(str).apply(map_numbers)
possible_paths_ref['Loading_time'] = handling_time

locations = node_list + mode_list
loc_to_index = {location: idx for idx, location in enumerate(locations)}

destinations = node_list
dest_to_index = {destination: idx for idx, destination in enumerate(destinations)}

d_profiles = ['no disruption'] + d_list
d_profile_to_index = {d_profile: idx for idx, d_profile in enumerate(d_profiles)}

def state_to_vector(state_s, state_to_index):
    vector = [0] * len(locations)
    vector[state_to_index[state_s]] = 1
    return vector

In [34]:
d_disruption_profile

[['Profile6', 'Volume', 0, 0, -0.3, 0.3, 2.1e-09]]

In [35]:
s_disruption_profile

[['Profile1', 'Delay', 3, 6, 0.0, 0.0, ['Train'], 0.00063],
 ['Profile2', 'Delay', 3, 9, 0.0, 0.0, ['Barge'], 0.00072],
 ['Profile3', 'Delay', 12, 48, 0.0, 0.0, ['Train', 'Barge'], 0.00012],
 ['Profile4', 'Delay', 3, 6, 0.0, 0.0, ['Terminal'], 0.00063],
 ['Profile5', 'Capacity reduction', 12, 24, 0.15, 0.2, ['Barge'], 0.00012]]

In [36]:
def shipment_logs(shipment_dict, actual_itinerary, actual_carried_shipments, assigned_to_rl, wait_actions, reassign_actions, late_data, episode):
    dfs = []

    # Iterate over each shipment in the dictionary
    for shipment in shipment_dict.values():
    # Calculate the total cost for the current shipment
        total_cost = (shipment.total_storage_cost +
                    shipment.total_handling_cost +
                    shipment.total_travel_cost +
                    shipment.total_delay_penalty)
        itinerary = actual_itinerary[shipment.name]
        if shipment.name in assigned_to_rl:
            rl = 'Yes'
        else:
            rl = 'No'
        missed_service = shipment.missed_service
        nr_wait = wait_actions[shipment.name]
        nr_reassign = reassign_actions[shipment.name]

        # Create a DataFrame for the current shipment
        df_shipment = pd.DataFrame({
            'Shipment': [shipment.name],
            'Storage Cost': [shipment.total_storage_cost],
            'Handling Cost': [shipment.total_handling_cost],
            'Travel Cost': [shipment.total_travel_cost],
            'Delay Penalty': [shipment.total_delay_penalty],
            'Total Cost': [total_cost],
            'Itinerary': [itinerary],
            'Assigned to RL': [rl],
            'Missed Service': [missed_service],
            'Wait Actions': [nr_wait],
            'Reassign Actions': [nr_reassign]
        })
        # Append the new DataFrame to the list
        dfs.append(df_shipment)

    # Concatenate all individual DataFrames into one
    df_shipment_costs = pd.concat(dfs, ignore_index=True)
    df_shipment_costs['Travel Cost'] = df_shipment_costs['Travel Cost'].round(2)
    df_shipment_costs['Handling Cost'] = df_shipment_costs['Handling Cost'].round(2)
    df_shipment_costs['Storage Cost'] = df_shipment_costs['Storage Cost'].round(2)
    df_shipment_costs['Delay Penalty'] = df_shipment_costs['Delay Penalty'].round(2)
    df_shipment_costs['Total Cost'] = df_shipment_costs['Total Cost'].round(2)

    # Crate service line dataframe
    df_service_line = pd.DataFrame(actual_carried_shipments.items(), columns=['Service Line', 'Number of Shipments'])
    late_data_df = pd.DataFrame.from_dict(late_data, orient='index', columns=['Late time', 'Number of late departure'])
    merged_df = df_service_line.merge(late_data_df, left_on='Service Line', right_index=True, how='left')

    # Export to csv
    # df_shipment_costs.to_csv(f'csv_output\Integrated_Model\Experiment 2 Output\V3\shipment_output_{policy_name}_{episode}.csv', index=False)
    # merged_df.to_csv(f'csv_output\Integrated_Model\Experiment 2 Output\V3\{sd}_20\service_line_output_{policy_name}_{episode}.csv', index=False)

In [37]:
epsilon = 0.05
# Q = defaultdict(lambda: np.zeros(n_actions))

# Define function to access Q-values safely
def get_q_value(Q, state, action, default_value=0):
    if state not in Q:
        Q[state] = {}
    if action not in Q[state]:
        Q[state][action] = default_value
    return Q[state][action]

# Initialize Q Table
def make_epsilon_greedy_policy(Q, epsilon, npA, mode_ID):
    def policy_fn(observation, possible_action):
        obs_tuple = tuple(observation)
        wait_ID = mode_ID[possible_action[0]]
        reassigned_ID = mode_ID[possible_action[1]]

        A = np.ones(npA, dtype=float) * epsilon / npA
        # best_action = np.argmax([Q[obs_tuple][wait_ID], Q[obs_tuple][reassigned_ID]])
        best_action = np.argmax([
                get_q_value(Q, obs_tuple, wait_ID),
                get_q_value(Q, obs_tuple, reassigned_ID)
            ])
        worse_action = 1 - best_action # for greedy policy

        #Epsilon-greedy policy
        print_event(f"Q[s,a] wait: {get_q_value(Q, obs_tuple, wait_ID)}, Q[s,a] reassign: {get_q_value(Q, obs_tuple, reassigned_ID)}")
        A[best_action] += (1.0 - epsilon)

        #Greedy policy
        A[best_action] = 1
        A[worse_action] = 0

        # Always reassign policy
        # A = [0,1]
        # policy_name = "Always reassign"

        # #Always wait policy
        # A = [1,0]
        # policy_name = "Always wait"

        return A
    return policy_fn

# A nested dictionary that maps state -> (action -> action-value).
n_actions = 1 + len(mode_list) # action in terminal state (0) and assigning to a service line
np_actions = 2 # wait or reassign

# Loading the Q-table
if os.path.exists(q_table_path):
    with open(f'{q_table_path}', 'rb') as f:
        Q = pickle.load(f)
        print(f'{q_table_path} is loaded')

else:
    Q = defaultdict(lambda: np.zeros(n_actions))
    print(f'New Q-table is created')
    
policy = make_epsilon_greedy_policy(Q, epsilon, np_actions, mode_ID)

q_table\q_table_200_50000_eps_v2 is loaded


In [38]:
## ----- Simulation ----- ##

def print_event(*args, **kwargs):
    if print_event_enabled:
        print(*args, **kwargs)

# Function to convert time to minutes
def time_format(minutes):
    return f"{(minutes%1440)//60:02d}:{(minutes%1440)%60:02d}"

# Function to represent the clock in the simulation
def clock(env, tick, simulation):
    while True:
        current_day = env.now // 1440 + 1
        print_event(" ")
        print_event(f"current day: {current_day}, simulation: {simulation + 1}")
        # print_event(f"List of pending shipment: {set(request['ID']) - set(delivered_shipments)}")
        yield env.timeout(tick)

def identify_truck_line(mode_name):
    name = ''
    for letter in mode_name:
        if letter != '.':
            name += letter
        else:
            break
    return name

def update_service_capacity(df, service_name, new_capacity):
    # Define a function to update capacities within a single row
    def update_row_capacities(service_ids, service_capacities, service_name, new_capacity):
        service_ids_list = service_ids.split(', ')
        service_capacities_list = service_capacities.split(', ')
        
        # Update the capacity for the matching service
        for i, service in enumerate(service_ids_list):
            if service == service_name:
                service_capacities_list[i] = str(new_capacity)
        
        # Join the updated capacities back into a string
        updated_capacities = ', '.join(service_capacities_list)
        return updated_capacities

    # Apply the function to each row in the dataframe
    df['service_capacities'] = df.apply(
        lambda row: update_row_capacities(row['service_ids'], row['service_capacities'], service_name, new_capacity), 
        axis=1
    )
    
    return df

# Define the mode of transport
class Mode:
    def __init__(self, env, name, schedule, capacity, speed, distance, costs):
        self.env = env
        self.name = name
        self.origin, self.destination, self.departure_time = schedule
        self.actual_departure = 0
        self.travel_cost1, self.travel_cost2, self.handling_cost = costs
        self.capacity = capacity
        self.free_capacity = capacity
        self.assigned_shipments = [] # for capacitated
        self.speed = speed
        self.distance = distance
        self.handling_time = handling_time # 1 minute for 1 container loading/unloading
        self.loading_time_window = 90 # Adjustable parameter
        self.loading = 0
        self.unloading = 0
        self.used_capacity = 0
        self.arrival = {node: env.event() for node in node_list}  # Events to signal when the barge/train arrives at a terminal
        self.handling_events = env.event()  # Event to signal when the barge finishes loading/unloading
        self.loading_events = env.event()
        self.departing_events = env.event()
        self.current_location = self.origin  # Track current location
        self.truck_service = env.event() # Event to signal the truck service
        self.status = "Available"

    # Function for vehicle operation
    def operate(self):
        
        global disruption_location
        global s_disruption_event
        global possible_paths
        global actual_carried_shipments
        global truck_name_list
        global rl_assignment
        global nr_late_departure
        global total_late_departure
        global late_logs
        global late_dict
        # Initialize the barge at the first location to load containers
        self.handling_events.succeed()
        
        while True:
            if 'Truck' in self.name:
                yield self.truck_service
            arrival_time = self.departure_time - self.loading_time_window
            operation_time = max(0, arrival_time - env.now - 120)
            yield env.timeout(operation_time)
            self.status = 'Operating'
            if arrival_time > env.now:
                yield env.timeout(arrival_time - env.now)  # Simulate first arrival 1,5 hr before the first departure
            if self.name in disruption_location:
                print_event(f'{time_format(env.now)} - {self.name} will arrive late in origin due to disruption at {self.name}')
                yield s_disruption_event[self.name]
            self.current_location = self.origin
            self.arrival[self.origin].succeed()
            print_event(f'{time_format(env.now)} - {self.name} is scheduled to depart from {self.origin} at {time_format(self.departure_time)}')
            # print_event(f'{time_format(env.now)} - TEST 1 {self.name} free capacity: {self.free_capacity} TEUs')
            # Simulate container loading time
            yield env.timeout(1)  # Wait for the used capacity to be updated
            self.arrival[self.origin] = env.event() # reset the event in origin location for next depature
            # prioritizing cargo with the earliest release time
            sorted_shipments = sorted(self.assigned_shipments, key=lambda x: x[1])
            filtered_list = [sublist[0] for sublist in self.assigned_shipments]
            rl_shipments = []

            for shipment in sorted_shipments:
                if shipment[0] not in rl_assignment: #Priorize undisrupted shipments
                    if self.used_capacity + shipment[2] <= self.capacity:
                        self.used_capacity += shipment[2]
                        self.loading += shipment[2]
                        filtered_list.remove(shipment[0])
                else:
                    rl_shipments.append(shipment)
            
            for shipment in rl_shipments:
                if self.used_capacity + shipment[2] <= self.capacity:
                    self.used_capacity += shipment[2]
                    self.loading += shipment[2]
                    filtered_list.remove(shipment[0])

            self.assigned_shipments = filtered_list
            self.loading_events.succeed()
            self.loading_events = env.event()

            yield env.timeout(self.handling_time * self.loading) # Simulate loading time
            self.assigned_shipments = [] #reset the assigned shipments

            # print_event(f'{time_format(env.now)} - TEST 2 {self.name} free capacity: {self.free_capacity} TEUs')
            if self.current_location in disruption_location:
                print_event(f'{time_format(env.now)} - {self.name} departure will be delayed due to disruption at {self.current_location}')
                yield s_disruption_event[self.current_location]
            # if self.loading > 0:
            print_event(f'{time_format(env.now)} - {self.name} finished loading {self.loading} TEUs at {self.origin}')
            # print_event(f'{time_format(env.now)} - TEST 3 {self.name} free capacity: {self.free_capacity} TEUs')
            # Wait until departure time
            if self.name in disruption_location:
                yield s_disruption_event[self.name]

            if env.now < self.departure_time:
                yield env.timeout(self.departure_time - env.now)
                # Signal loading done
                if self.current_location in disruption_location:
                    if self.used_capacity > 0: #to focus the output on used mode
                        print_event(f'{time_format(env.now)} - {self.name} departure will be delayed due to disruption at {self.current_location}')
                    yield s_disruption_event[self.current_location]
                self.actual_departure = env.now
                if self.used_capacity > 0:
                    print_event(f'{time_format(env.now)} - {self.name} departs from {self.origin} carrying {self.used_capacity} TEUs')
            else:
                self.actual_departure = env.now
                if self.used_capacity > 0:
                    print_event(f'{time_format(env.now)} - {self.name} late departure from {self.origin} with free capacity {self.capacity - self.used_capacity} TEUs')
                    late_departure = self.actual_departure - self.departure_time
                    total_late_departure += late_departure
                    nr_late_departure += 1
                    if 'Truck' in self.name:
                        name = identify_truck_line(self.name)
                    else:
                        name = self.name
                    late_logs.append([name, late_departure])
                    late_dict[name][0] += late_departure
                    late_dict[name][1] += 1

            # print_event(f'{time_format(env.now)} - TEST 4 {self.name} free capacity: {self.free_capacity} TEUs')
            self.departing_events.succeed()
            self.departing_events = env.event()
            self.handling_events = env.event()
            self.loading = 0 # Reset the loading counter
            self.current_location = self.name
            self.status = "En route"
            possible_paths.loc[possible_paths['first_service'] == self.name, 'first_service_departure'] += 168
            possible_paths.loc[possible_paths['first_service'] == self.name, 'last_service_arrival'] += 168
            
            # Travel to destination
            if self.name in disruption_location:
                # print_event(f'{time_format(env.now)} - {self.name} will be late due to disruption on the service line')
                yield s_disruption_event[self.current_location]
            yield env.timeout(int(self.distance / self.speed * 60))
            
            
            # Signal the change of location for arrival
            self.current_location = self.destination
            if self.current_location in disruption_location:
                print_event(f'{time_format(env.now)} - {self.name} arriving late at {self.destination} due to disruption at {self.current_location}')
                yield s_disruption_event[self.current_location]

            self.arrival[self.destination].succeed()
            print_event(f'{time_format(env.now)} - {self.name} arrived at {self.destination}')
            
            # Simulate container unloading time
            yield env.timeout(1)  # Wait for the used capacity to be updated
            yield env.timeout(self.handling_time * self.unloading) # Simulate unloading time
            
            # For observation
            if 'Truck' in self.name:
                name = identify_truck_line(self.name)
                actual_carried_shipments[name] = self.unloading
            else:
                actual_carried_shipments[self.name] += self.unloading

            if self.unloading > 0:
                print_event(f'{time_format(env.now)} - {self.name} finished unloading {self.unloading} TEUs at {self.destination}')
            self.unloading = 0 # Reset the unloading counter
            
            # Signal unloading done
            self.handling_events.succeed()
            self.arrival[self.destination] = env.event() #reset the event in destination location for next leg
            self.status = "Available"
            if 'Truck' in self.name:
                self.truck_service = env.event() #finish the truck service
                self.departure_time = 99999 #reset the truck's departure time
                truck_name_list.remove(self.name)
            else:
                self.departure_time = self.departure_time + 7*1440  # Wait for the next week
                # possible_paths.loc[possible_paths['first_service'] == self.name, 'first_service_departure'] += 168
                # possible_paths.loc[possible_paths['first_service'] == self.name, 'last_service_arrival'] += 168

# Define shippement
class Shipment:
    def __init__(self, env, request_details):
        self.env = env
        self.name, self.origin, self.destination, release_time, due_time, self.num_containers, self.mode, self.possible_itineraries, announce_time = request_details
        self.announce_time = announce_time * 60
        self.release_time = release_time * 60
        self.due_time = due_time * 60
        self.loading = env.event()
        self.total_storage_cost = 0
        self.total_handling_cost = 0
        self.total_travel_cost = 0
        self.total_delay_penalty = 0 
        self.current_location = self.origin
        self.planning = env.event()
        self.process = env.process(self.handled())
        self.status = "Announced"
        self.loading_signal = env.event()
        self.assigned_to_rl = False
        self.rl_start_time = 0
        #for RL
        self.reward = 0
        self.total_reward = 0
        self.reward_event = env.event()
        self.state_event = {mode: env.event() for mode in (mode_list)}
        self.action_event = {mode: env.event() for mode in (mode_list)}
        self.missed_service = 0
    
    def handled(self):

        #set global variables
        global total_storage_time
        global total_storage_cost
        global total_handling_cost
        global total_travel_cost
        global total_shipment_delay
        global total_delay_penalty
        global announced_requests
        global active_requests
        global unassigned_requests
        global requests_to_replan
        global affected_requests
        global storage_time_list
        global truck_waiting_time
        global total_cost
        global actual_itinerary

        n = 0
        yield env.timeout(self.announce_time)
        print_event(f"{time_format(env.now)} - {self.name} with {self.num_containers} containers requests transport from {self.origin} to {self.destination}")
        announced_requests.append(self.name)
        
        unassigned_requests.append([self.name, self.origin, self.destination, self.release_time, self.due_time, self.num_containers, self.mode])
        active_requests.append(self.name)

        #set event to wait for mode assignments, wait until planning period
        while self.status != "Assigned":
            try:
                yield self.planning
                self.status = "Assigned"
            except sim.Interrupt:
                if self.status == "New Release Time":
                    print_event(f"{time_format(env.now)} - {self.name} has a new release time ({self.release_time})")
                elif self.status == "New Volume":
                    print_event(f"{time_format(env.now)} - {self.name} has new container volume: {self.num_containers} TEUs")
        
        #update mode free capacity
        # for mode in self.mode:
        #     mode.free_capacity -= self.num_containers
        
        for req in unassigned_requests:
            if req[0] == self.name:
                unassigned_requests.remove(req)
        
        #wait until release time
        while self.release_time > env.now:
            try:
                yield env.timeout(self.release_time - env.now)
            except sim.Interrupt:
                if self.status == "New Release Time":
                    print_event(f"{time_format(env.now)} - {self.name} has a new release time ({self.release_time})")
                    if self.release_time > self.mode[n].departure_time:
                        print_event(f"{time_format(env.now)} - {self.name} is assigned to {self.mode[n].name} with departure time {self.mode[n].departure_time}")
                        print_event(f"{time_format(env.now)} - {self.name} will miss the service")
                        for i in range(len(self.mode)):
                            # self.mode[i].status = "Available"
                            self.mode[i] = self.mode[i].name
                        self.release_time = max(env.now, self.release_time)
                        requests_to_replan.append([self.name, self.origin, self.destination, self.release_time, self.due_time, self.num_containers, self.mode])
                        disruption_location.append(self.name)
                        planning.replanning()
                        disruption_location.remove(self.name)

                elif self.status == "New Volume":
                    print_event(f"{time_format(env.now)} - {self.name} has new container volume: {self.num_containers} TEUs")
                    if self.num_containers > self.mode[n].free_capacity:
                        print_event(f"{time_format(env.now)} - {self.name} cant be assigned to {self.mode[n].name} due to insufficient capacity")
                        for i in range(len(self.mode)):
                            # self.mode[i].status = "Available"
                            self.mode[i] = self.mode[i].name
                        requests_to_replan.append([self.name, self.origin, self.destination, self.release_time, self.due_time, self.num_containers, self.mode])
                        disruption_location.append(self.name)
                        planning.replanning()
                        disruption_location.remove(self.name)
                    else:
                        self.mode[n].free_capacity -= self.num_containers
                        
        announced_requests.remove(self.name)

        # Simulate the shipment handling
        while self.current_location != self.destination:
            self.status = "Waiting for arrival"
            # order truck if the mode is truck
            if 'Truck' in self.mode[n].name:
                self.mode[n].truck_service.succeed() #trigger the truck service
                self.mode[n].departure_time = self.release_time + truck_waiting_time
                
            print_event(f"{time_format(env.now)} - {self.name} will be transported from {self.current_location} to {self.mode[n].destination} on {self.mode[n].name}")
            while self.status == "Waiting for arrival":
                try:
                    # Wait for the mode to arrive at the shipment's origin
                    yield self.mode[n].arrival[self.current_location]

                    # For capacitated
                    self.mode[n].assigned_shipments.append([self.name, self.release_time, self.num_containers])
                    yield self.mode[n].loading_events
                    if self.name in self.mode[n].assigned_shipments:
                        print_event(f"{time_format(env.now)} - {self.name} wait for the next arrival for mode {self.mode[n].name} due to insufficient capacity")
                        self.missed_service += 1
                        yield self.mode[n].arrival[self.mode[n].destination]
                    else:
                        self.status = "Ready to load"

                except sim.Interrupt:
                    if self.current_location != self.mode[n].destination:
                        print_event(f"{time_format(env.now)} - {self.name} is replanned and will be transported from {self.current_location} to {self.mode[n].destination} on {self.mode[n].name}")
                    if 'Truck' in self.mode[0].name:
                        self.mode[n].truck_service.succeed() #trigger the truck service
                        self.mode[n].departure_time = env.now + truck_waiting_time
                        # self.mode[n].used_capacity += self.num_containers
                        
            # Simulate loading containers onto the mode
            print_event(f"{time_format(env.now)} - {self.name} starts loading on {self.mode[n].name}")
            # print(f'TEST DEBUG RELEASE TIME: {self.release_time}')
            self.finish_loading = env.now
            storage_time = max(0, env.now - self.release_time) # Calculate the storage time
            shipment_storage_cost = (storage_time / 60) * storage_cost * self.num_containers # Calculate the storage cost
            total_storage_time += storage_time # Calculate the total storage time for all shipments
            self.total_storage_cost += shipment_storage_cost
            total_storage_cost += shipment_storage_cost # Calculate the total storage cost for all shipments
            # print(f'TEST DEBUG STORAGE COST: {shipment_storage_cost}')
            
            # Calculate reward for RL (storage)
            if self.assigned_to_rl:
                storage_time_rl = max(0, env.now - self.rl_start_time)
                self.reward += ((storage_time_rl / 60) * storage_cost * self.num_containers) * -1
                # print(f'{time_format(env.now)} - TEST {self.name} got reward: {self.reward} for storage')

            # Calculate the loading cost
            loading_cost = self.num_containers * self.mode[n].handling_cost
            self.total_handling_cost += loading_cost
            total_handling_cost += loading_cost

            # Calculate reward for RL (loading)
            if self.assigned_to_rl:
                self.reward += (self.num_containers * self.mode[n].handling_cost) * -1
                # print(f'{time_format(env.now)} - TEST {self.name} got reward: {self.reward} + loading')
            
            # Simulate travel time from origin to destination
            self.current_location = self.mode[n].name
            self.status = "On board"
            yield self.mode[n].departing_events  # Wait for until mode actually departs
            # calculate extra storage time while idling on board before actual departure
            extra_storage_time = max(0, env.now - self.finish_loading)
            extra_storage_cost = (extra_storage_time / 60) * storage_cost * self.num_containers
            total_storage_time += extra_storage_time
            self.total_storage_cost += extra_storage_cost
            total_storage_cost += extra_storage_cost

            if self.assigned_to_rl:
                # storage_time_rl = max(0, env.now - self.rl_start_time)
                self.reward += extra_storage_cost * -1
                # print(f'{time_format(env.now)} - TEST {self.name} got reward: {self.reward} extra storage')

            #update mode free capacity
            self.mode[n].free_capacity += self.num_containers

            # Update possible itineraries
            updated_itinerary = []
            if self.possible_itineraries:
                for path in self.possible_itineraries:
                    if path[0] == self.mode[n].name:
                        updated_itinerary.append(path[1:])
            self.possible_itineraries = updated_itinerary
            
            yield self.mode[n].arrival[self.mode[n].destination]
            
            travel_cost1 = self.mode[n].travel_cost1 * (env.now - self.mode[n].actual_departure)/60 *self.num_containers
            travel_cost2 = self.mode[n].travel_cost2 * self.mode[n].distance * self.num_containers
            travel_cost = travel_cost1 + travel_cost2
            self.total_travel_cost += travel_cost
            total_travel_cost += travel_cost

            # Calculate reward for RL (travel)
            if self.assigned_to_rl:
                self.reward += travel_cost * -1 #reward for previous action
                # print(f'{time_format(env.now)} - TEST {self.name} got reward: {self.reward} for + travel')

            # Simulate unloading containers
            self.mode[n].used_capacity -= self.num_containers
            self.mode[n].unloading += self.num_containers
            yield self.mode[n].handling_events  # Wait for until unloading is done
            print_event(f"{time_format(env.now)} - {self.name} completed unloading at {self.mode[n].destination} on {self.mode[n].name}")

            #calculate the handling cost
            unloading_cost = self.num_containers * self.mode[n].handling_cost
            self.total_handling_cost += unloading_cost
            total_handling_cost += unloading_cost

            # Calculate reward for RL (unloading)
            if self.assigned_to_rl:
                self.reward += (self.num_containers * self.mode[n].handling_cost) * -1
                # print(f'{time_format(env.now)} - TEST {self.name} got reward: {self.reward} for + unloading')
                
            self.current_location = self.mode[n].destination
            self.release_time = env.now

            # Check if the shipment is assigned to RL during travelling
            self.total_reward += self.reward
            # print_event(f"TEST - {self.name} current reward {self.reward}")
            if self.assigned_to_rl:
                if self.mode[n] != self.mode[-1]: # Check if the shipment is on the last mode
                    self.rl_start_time = env.now # Update start time for next reward calculation
                    self.reward_event.succeed()

                    # Trigger RL to update state for next action
                    if 'Truck' in self.mode[n+1].name:
                        name = identify_truck_line(self.mode[n+1].name)
                        self.state_event[name].succeed()
                        # self.action_event[name].succeed()
                    else:
                        self.state_event[self.mode[n+1].name].succeed()
                        # self.action_event[self.mode[n].name].succeed()
                    # print(f'TEST {self.name} update state for {self.mode[n+1].name}')

                    # Trigger RL to update action event once the action is completed
                    if 'Truck' in self.mode[n].name:
                        name = identify_truck_line(self.mode[n].name)
                        self.action_event[name].succeed()
                    else:
                        self.action_event[self.mode[n].name].succeed()
                    # print(f'TEST {self.name} trigger action event for {self.mode[n].name}')

                    yield env.timeout(1)
                    self.reward_event = env.event() #reset the reward event for next action
                else:
                    if 'Truck' in self.mode[n].name:
                        name = identify_truck_line(self.mode[n].name)
                        self.action_event[name].succeed()
                    else:
                        self.action_event[self.mode[n].name].succeed()

            if self.name in rl_assignment and not self.assigned_to_rl:
                self.assigned_to_rl = True #triggered if the shipment is assigned to RL during travelling
                self.rl_start_time = env.now
                # self.reward_event.succeed()
                # Trigger RL to update state for next action
                if 'Truck' in self.mode[n+1].name:
                    name = identify_truck_line(self.mode[n+1].name)
                    self.state_event[name].succeed()
                    # self.action_event[name].succeed()
                    # self.state_event[name] = env.event()
                else:
                    self.state_event[self.mode[n+1].name].succeed()
                    # self.action_event[self.mode[n].name].succeed()
                    # self.state_event[self.mode[n].name] = env.event()
                # self.state_event.succeed()
                # self.state_event = env.event()
                # self.reward_event = env.event()

            print_event(f"{time_format(env.now)} - {self.name} is available at {self.current_location}")
            actual_itinerary[self.name].append(self.mode[n].name)
            self.mode.pop(0) # Remove the completed service from the itinerary
            # n += 1
        self.status = "Delivered"
        print_event(f"{time_format(env.now)} - {self.name} has been delivered to {self.destination}")
        delivered_shipments.append(self.name)
        active_requests.remove(self.name)

        if env.now > self.due_time:
            delay = env.now - self.due_time
            shipment_delay_penalty = (delay / 60) * delay_penalty * self.num_containers
            print_event(f"{time_format(env.now)} - {self.name} is late for {delay//60:02d} hour(s) {delay%60:02d} minute(s)")
            total_shipment_delay += delay
            self.total_delay_penalty += shipment_delay_penalty
            total_delay_penalty += shipment_delay_penalty

            # Calculate reward for RL (delay)
            if self.assigned_to_rl:
                self.reward += shipment_delay_penalty * -1
                # print(f'{time_format(env.now)} - TEST {self.name} got reward: {self.reward} + delay')
                self.reward_event.succeed()
                yield env.timeout(1)
                rl_assignment.remove(self.name)
        else:
            if self.assigned_to_rl:
                self.reward_event.succeed()
                rl_assignment.remove(self.name)

        total_cost += self.total_storage_cost + self.total_handling_cost + self.total_travel_cost + self.total_delay_penalty
        self.total_reward += self.reward
        storage_time_list.append(storage_time)

# Function to check for disrupted requests
def affected_request_detection(env, shipment, s_disruption, planning):
    while True:
        # Wait until a service disruption occurs
        yield s_disruption.disruption_signal
        new_disrupted_location = disruption_location[-1]
        affected_requests_list = [] # Initiate a list of affected requests for the new disruption
        for request in active_requests:
            locations = [] # Initiate a list of locations in the shipment's itinerary
            if shipment[request].mode:
                if not isinstance (shipment[request].mode[0], str):
                    for mode in shipment[request].mode:
                        locations.append(mode.name)
                        locations.append(mode.destination) # Add the assigned mode's destination
            if shipment[request].current_location in locations:
                locations.remove(shipment[request].current_location) # Remove the current location
            if shipment[request].current_location in mode_list:
                locations.remove(shipment[request].mode[0].destination) # Remove the current location destination if a shipment is on a service line
            end_destination = shipment[request].destination
            if end_destination in locations:
                locations.remove(end_destination) # Remove the end destination
            # Check if the disrupted location is in the shipment's itinerary
            if locations:
                # print(f"TEST {locations} and new disruption {new_disrupted_location}")
                if any(location in new_disrupted_location for location in locations):
                    affected_requests_list.append(request)
        print_event(f"{time_format(env.now)} - Affected requests: {affected_requests_list}")
        affected_requests[new_disrupted_location] = affected_requests_list

        for request in affected_requests_list:
            s = shipment[request]
            if s.current_location in node_list:
                s.origin = s.current_location
            else:
                s.origin = s.mode[0].destination
                print_event(f"{time_format(env.now)} - {s.name} is is disrupted on board")
            for i in range(len(s.mode)):
                s.mode[i].status = "Available"
                s.mode[i].free_capacity += s.num_containers
                s.mode[i] = s.mode[i].name
            # s.release_time = max(env.now, s.release_time)
            # requests_to_replan.append([s.name, s.origin, s.destination, s.release_time, s.due_time, s.num_containers, s.mode])
            requests_to_replan.append([s.name, s.origin, s.destination, s.release_time, s.due_time, s.num_containers, s.mode])
        # IF the request to replan is not empty, trigger the replanning process
        if requests_to_replan:
            planning.replanning()
        s_disruption.disruption_signal = env.event()

# Matching Module
class MatchingModule():

    def __init__(self, env, mode_schedule, shipment, rl_module):
        self.env = env
        # self.combinations = combinations
        # self.possible_paths = possible_paths #for optimization module input
        self.mode_schedule = mode_schedule
        self.shipment = shipment
        self.disruption_event = env.event()
        self.rl_module = rl_module

    def planning(self):

        yield env.timeout(1)
        while True:
            # print("\nPLANNING") # TO EXTRACT FOR OPTIMIZATION MODULE
            disrupted_location = disruption_location
            print_event(f"{time_format(env.now)} - disruption at {disrupted_location}")
            request_list = unassigned_requests
            # For separate planning
            planned_requests = []
            unplanned_requests = []
            for req in request_list:
                if not req[6]:
                    unplanned_requests.append(req)
                else:
                    planned_requests.append(req)
            unplanned_requests
#---------------------------------OPTIMIZATION MODULE---------------------------------
            # Activate this if the optimization module uses df as input
            available_paths = self.FilterPath()

            # available_paths.to_csv(f"{path}\\available_paths.csv", index=False)
            if request_list:
                matching = self.OptimizationModule(unplanned_requests, available_paths)
#------------------------------------------------------------------------------------

            # available_combinations = self.FilterCombination()
            # matching = self.SimpleHeuristic(request_list, available_combinations)

            for req in planned_requests:
                matching[req[0]] = ([], req[6])
            self.ModeAssignment(request_list, matching)
            yield env.timeout(7*1440)
            # self.combinations = [list[0:5] + [list[5] + (7*1440)] for list in self.combinations] # Update the departure time for the next week

    def replanning(self):

        global requests_to_replan
        global d_profile_list
        global rl_assignment
        global reward_generator
        global rg_order
        global rl_triggers
        global assigned_to_rl
        global truck_name_list

        disrupted_location = disruption_location[-1]
        print_event(f"{time_format(env.now)} - Replanning due to disruption at {disrupted_location}")
        request_list = requests_to_replan
        if disrupted_location in node_list or disrupted_location in mode_list: # To skip this process for disruption in request
            for request in request_list:
                if self.shipment[request[0]].current_location in node_list:
                    self.shipment[request[0]].process.interrupt()
                    self.shipment[request[0]].planning = env.event() #Prevent double succeed triggers
                else:
                    self.shipment[request[0]].planning = env.event() #Prevent double succeed triggers
        else:
            self.shipment[request_list[0][0]].planning = env.event()

        available_paths = self.FilterPath()

        # available_paths.reset_index(drop=True, inplace=True)
        # available_paths.to_csv(f"{path}\\available_paths.csv", index=False)
        unsolved_requests = []
        solved_requests = []
        for req in request_list:
            available_next_solution = []
            for path in self.shipment[req[0]].possible_itineraries:
                if disrupted_location not in path:
                    available_next_solution.append(path)
            if not available_next_solution:
                unsolved_requests.append(req)
            else:
                for path in available_next_solution:
                    path_capacity = []
                    for mode in path:
                        if 'Truck' not in mode:
                            path_capacity.append(self.mode_schedule[mode].free_capacity)
                        else:
                            path_capacity.append(99999)
                    path_capacity = min(path_capacity)
                    if req[5] <= path_capacity:
                        solved_requests.append(req)
                        new_mode = path
                        req[6] = [req[6], new_mode]
                        break
                if req not in solved_requests:
                    unsolved_requests.append(req)

        # print(f"DEBUG UNSOLVED REQUESTS: {unsolved_requests}")
        # print(f"DEBUG SOLVED REQUESTS: {solved_requests}")
        matching = self.OptimizationModule(unsolved_requests, available_paths)
        for req in solved_requests:
            new_mode = req[6][1]
            old_mode = req[6][0]
            matching[req[0]] = (old_mode, new_mode)

        # Assign the unmatched requests to truck
        for req in request_list:
            # print(f'DEBUG {req[0]} - {matching[req[0]]}')
            if not matching[req[0]][1]:
                origin = self.shipment[req[0]].origin
                destination = self.shipment[req[0]].destination
                old_mode = matching[req[0]][0]
                new_mode = old_mode
                for mode in truck_list:
                    if truck_schedule_dict[mode][1][0] == origin and truck_schedule_dict[mode][1][1] == destination:
                        new_mode = [mode]
                        break
                matching[req[0]] = (old_mode, new_mode)

        # available_combinations = self.FilterCombination()
        # matching = self.SimpleHeuristic(request_list, available_combinations)

        if disrupted_location in node_list or disrupted_location in mode_list or disrupted_location in truck_name_list: # Triggers RL only if service disruption
            #RL algorithm
            rl_triggers += 1
            rl_match = {}
            # print(f'rl assignment: {rl_assignment}') #debug
            for request in request_list:
                if request[0] not in assigned_to_rl:
                    assigned_to_rl.append(request[0])
                
                #actions
                action_sets = matching[request[0]]

                # states
                current_location = self.shipment[request[0]].current_location
                destination = self.shipment[request[0]].destination
                due_time = self.shipment[request[0]].due_time // 60
                volume = self.shipment[request[0]].num_containers
                d_profile = d_profile_list[-1][0]
                current_time = env.now % (1440*7) // 60
                current_state = current_location, destination, due_time, volume, d_profile, current_time
                print_event(f'State for RL: {current_state}, Actions: {action_sets}')

                ## Notes
                # actions[0] = wait, actions[1] = reassign
                # reward will be delayed untul the shipment arrives at the next terminal
                action_set_taken = self.rl_module.action_generator(request, current_state, action_sets)
                

                if request[0] not in rl_assignment:
                    #Setup first assignment to RL
                    reward_generator[request[0]] = []
                    if self.shipment[request[0]].status == "Waiting for arrival":
                        self.shipment[request[0]].rl_start_time = env.now
                        self.shipment[request[0]].assigned_to_rl = True
                    elif self.shipment[request[0]].status == "On board":
                        self.shipment[request[0]].assigned_to_rl = False #wait until arrrive in the next terminal
                    else:
                        self.shipment[request[0]].rl_start_time = max(env.now, self.shipment[request[0]].release_time)
                        self.shipment[request[0]].assigned_to_rl = True

                else:
                    # Update reward from previous actions
                    if self.shipment[request[0]].status == "Waiting for arrival":
                        storage_time_rl = max(0, env.now - self.shipment[request[0]].rl_start_time)
                        self.shipment[request[0]].reward += ((storage_time_rl / 60) * storage_cost * self.shipment[request[0]].num_containers) * -1

                    for process_ID in reward_generator[request[0]]:
                        process = process_ID[0]
                        process.interrupt()
                        
                    reward_generator[request[0]] = []
                    self.shipment[request[0]].rl_start_time = env.now
                    self.shipment[request[0]].assigned_to_rl = True
            
                # Generate initial state for each action
                for i in range(len(action_set_taken)):
                    print_event(f"RL ASSIGNMENT: {request[0]} - {action_set_taken[i]}")
                    if i == 0:
                        state = current_state
                        future = False
                    else:
                        state = current_state
                        future = True
                    rg_order += 1
                    # in case action taken is wait from previous action
                    if 'Truck' in action_set_taken[i]:
                        action_set_taken[i] = identify_truck_line(action_set_taken[i])
                    reward_gen = env.process(self.rl_module.reward_generator(request, state, action_set_taken[i], future, rg_order))
                    reward_generator[request[0]].append([reward_gen, rg_order])
                    
                rl_assignment.append(request[0]) 

                rl_match[request[0]] = (action_sets[0], action_set_taken)
            self.ModeAssignment(request_list, rl_match)
        else:
            self.ModeAssignment(request_list, matching)
        requests_to_replan = [] #reset the requests to replan


# Optimization Model----------------------------------------------
    
    def FilterPath(self):

        disrupted_location = disruption_location
        available_paths = possible_paths[:]
        # available_paths.to_csv(f"{path}\\available_paths.csv", index=False)
        for location in disrupted_location:
            if location != 0:
                if location in mode_list: # For disruption in service line
                    available_paths = available_paths[~available_paths['service_ids'].str.contains(location)]
                else: # For disruption in destination
                    available_paths = available_paths[~available_paths['Transshipment Terminal(s)'].str.contains(location)]
        return available_paths

    def OptimizationModule(self, request_list, available_paths):
        global om_triggers

        om_triggers += 1
        print_event(f'This is triggers number: {om_triggers}')

        # Convert input to df
        df_request_list = pd.DataFrame(request_list, columns = ['Demand_ID', 'Origin', 'Destination', 'Release Time', 'Due Time', 'Container Volume', 'Mode'])
        # df_request_list.to_csv(f"{path}\\Integrated Model\\om_request{om_triggers}.csv", index=False)

        # Demand input preprocessing
        df_request_list['Release Time'] = df_request_list['Release Time'] / 60
        df_request_list['Due Time'] = df_request_list['Due Time'] / 60

        # Add dummy announce time
        # df_request_list = df_request_list.reset_index()
        # df_request_list = df_request_list.rename(columns = {'index': 'Announce Time'})

        # Service input preprocessing
        capacitated_service = available_paths.copy()
        for service in fixed_list:
            free_capacity = self.mode_schedule[service].free_capacity
            capacitated_service.loc[capacitated_service['service_ids'].str.contains(service), 'service_capacity'] = capacitated_service.loc[capacitated_service['service_ids'].str.contains(service), 'service_capacity'].apply(lambda x: max(0, (min(x, free_capacity))))
            update_service_capacity(capacitated_service, service, free_capacity)
        # capacitated_service.to_csv(f"{path}\\Integrated Model\\om_available_paths_{om_triggers}.csv", index=False)

        # df_request_list.to_csv(f"{path}\\Integrated Model\\om_request_{om_triggers}.csv", index=False)
        capacitated_service['Loading Time'] = handling_time / 60
        matching = om.run_optimization(df_request_list, capacitated_service, storage_cost, delay_penalty, penalty_per_unfulfilled_demand)
        df_matching = pd.DataFrame(matching)
        # df_matching.to_csv(f"{path}\\Integrated Model\\om_matching.csv", index=False)
        if df_matching.empty:
            df_matching_combined = df_request_list
            df_matching_combined['Service_ID'] = 0
        else:
            df_matching_combined = df_request_list.merge(df_matching, on='Demand_ID', how='left').fillna(0)
        # df_matching_combined.to_csv(f"{path}\\om_matching_combined.csv", index=False)
        matching_result = dict(zip(df_matching_combined['Demand_ID'], df_matching_combined['Service_ID']))

        for key, values in matching_result.items():
            old_mode = df_request_list.loc[df_request_list['Demand_ID'] == key, 'Mode'].values[0]
            for i in range(len(old_mode)):
                if 'Truck' in old_mode[i]:
                    old_mode[i] = identify_truck_line(old_mode[i])
            if values == 0:
                new_mode = []
                print_event(f"{time_format(env.now)} - {key} has no possible new mode assignment")
            else:
                new_mode = [values]
                new_mode = [item.strip() for sublist in new_mode for item in sublist.split(',')]

            matching_result[key] = (old_mode, new_mode)

        return matching_result
        # output will be new itinerary for each request

# ----------------------------------------------------------------
    
    def ModeAssignment(self, request_list, match):
        global truck_id
        global truck_name_list

        for request in request_list:

            current_location = self.shipment[request[0]].current_location
            old_mode, new_mode = match[request[0]]
            assigned_mode = []
            if new_mode:
                if current_location in mode_list:
                    assigned_mode = [self.mode_schedule[current_location]]
                
                if old_mode == new_mode:
                    print_event(f"{time_format(env.now)} - {request[0]} from {request[1]} to {request[2]} will wait")
                    assigned_mode = []
                else:
                    print_event(f"{time_format(env.now)} - {request[0]} from {request[1]} to {request[2]} is assigned to {new_mode}")

                # for mode in old_mode:
                #     self.mode_schedule[mode].status = "Available"
                
                for mode in new_mode:
                    
                    if 'Truck' in mode:
                        if len(mode) < 8:
                            truck_id += 1
                            name, schedule, capacity, speed, distance, costs = truck_schedule_dict[mode]
                            name = f'{mode}.{truck_id}'
                            self.mode_schedule[name] = Mode(env, name, schedule, capacity, speed, distance, costs)
                            truck_name_list.append(name)
                            env.process(self.mode_schedule[name].operate())
                            # assigned_mode.append(self.mode_schedule[name])
                        else:
                            name = mode
                        assigned_mode.append(self.mode_schedule[name])
                        # self.mode_schedule[name].status = "Occupied"
                    else:
                        assigned_mode.append(self.mode_schedule[mode])
                        # self.mode_schedule[mode].status = "Occupied"

                self.shipment[request[0]].mode = assigned_mode
                for mode in assigned_mode:
                    mode.free_capacity -= self.shipment[request[0]].num_containers
                self.shipment[request[0]].planning.succeed()
            
            else:
                # print_event(f"{time_format(env.now)} - {request[0]} has no possible new mode assignment")
                new_mode = old_mode
                if new_mode:
                    print_event(f"{time_format(env.now)} - {request[0]} from {request[1]} to {request[2]} will wait")
                    for mode in new_mode:
                        assigned_mode.append(self.mode_schedule[mode])
                        # self.mode_schedule[mode].status = "Occupied"
                    self.shipment[request[0]].mode = assigned_mode
                    for mode in assigned_mode:
                        mode.free_capacity -= self.shipment[request[0]].num_containers
                    self.shipment[request[0]].planning.succeed()
                    
# Reinforcement Learning Module
class ReinforcementLearning():

    def __init__(self, env, shipment, mode_schedule, q_table, discount_factor, alpha):
        self.env = env
        self.shipment = shipment
        self.mode_schedule = mode_schedule
        self.q_table = q_table
        self.discount_factor = discount_factor
        self.alpha = alpha
        self.generate_reward = env.event()
        self.queue = []
        #TEST
        self.test_queue = []
        # env.process(self.update_q_table(self.q_table, self.discount_factor, self.alpha))

    def action_generator(self, request, state, action_sets):

        global wait_actions
        global reassign_actions

        wait = action_sets[0][0]
        if 'Truck' in wait:
            wait = identify_truck_line(wait)
        immediate_action = action_sets[1][0]
        possible_action = [wait, immediate_action]

        # Convert state to vector
        current_location_vector = tuple(self.state_to_vector(state[0], loc_to_index))
        destination_vector = tuple(self.state_to_vector(state[1], dest_to_index))
        profile_vector = tuple(self.state_to_vector(state[4], d_profile_to_index))
        state_vector = current_location_vector, destination_vector, state[2], state[3], profile_vector, state[5]

        action_probs = policy(state_vector, possible_action)
        vary_seed = np.random.default_rng(int(time.time() * 1000))
        action_ID = vary_seed.choice(np.arange(len(action_probs)), p=action_probs)
        chosen_set = action_sets[action_ID]
        non_chosen = action_sets[1 - action_ID]

        # for mode in non_chosen:
        #     self.mode_schedule[mode].status = "Available"
        if action_ID == 0:
            print_event(f'{time_format(env.now)} - RL choose wait for {request[0]}')
            wait_actions[request[0]] += 1
        else:
            print_event(f'{time_format(env.now)} - RL choose reassign for {request[0]}')
            reassign_actions[request[0]] += 1
        
        return chosen_set

    def reward_generator(self, request, state, action, future, gen_order):
        global total_reward
        global rl_number # debug
        self.function_stop = env.event()
        # self.shipment[request[0]].reward_event = env.event()

        order = rl_number + 1
        rl_number += 1
        action_taken = mode_ID[action]
        try: 
            if not future:
                updated_state = state
                wait = False
            else:
                wait = True
                # yield self.shipment[request[0]].state_event
                if 'Truck' in action:
                    yield self.shipment[request[0]].state_event[action]
                    # print(f'UPDATE STATE {time_format(env.now)} - {request[0]} for action: {action}')
                else:
                    yield self.shipment[request[0]].state_event[action]
                    # print(f'UPDATE STATE {time_format(env.now)} - {request[0]} for action: {action}')
                wait = False
                current_location = self.shipment[request[0]].current_location
                destination = self.shipment[request[0]].destination
                due_time = self.shipment[request[0]].due_time // 60
                volume = self.shipment[request[0]].num_containers
                d_profile = 'no disruption'
                current_time = env.now % (1440*7) // 60
                updated_state = current_location, destination, due_time, volume, d_profile, current_time
            
            # print(f'TEST reward generator {time_format(env.now)} - {request[0]} WAIT FOR REWARD EVENT for ACTION {action}')
            yield self.shipment[request[0]].reward_event
            # print(f'TEST reward generator {time_format(env.now)} - {request[0]} FINISH WAITING for ACTION {action}')

            if 'Truck' in action:
                yield self.shipment[request[0]].action_event[action]
                # print(f'TEST trigger action event {time_format(env.now)} - {request[0]} takes action: {action}')
            else:
                # print(f'IS IT HERE {action} for {request[0]}')
                yield self.shipment[request[0]].action_event[action]
                # print(f'TEST trigger action event {time_format(env.now)} - {request[0]} takes action: {action}')
            
            # # Determine next action
            # if self.shipment[request[0]].status == "Delivered":
            #     next_action = 0 # no action after a shipment is delivered
            # else:
            #     mode = self.shipment[request[0]].mode[0].name
            #     if 'Truck' in mode:
            #         name = identify_truck_line(mode)
            #         next_action = mode_ID[name]
            #     else:
            #         next_action = mode_ID[mode]

            # Determine next state
            current_location, destination, due_time, volume, d_profile, current_time = updated_state
            current_location = self.shipment[request[0]].current_location
            d_profile = 'no disruption'
            current_time = env.now % (1440*7) // 60
            next_state = current_location, destination, due_time, volume, d_profile, current_time
            # print(f'debug next state: {next_state}')

        except sim.Interrupt:
            if wait:
                yield self.function_stop
            
            if self.shipment[request[0]].status == "On board":
                # wait until it arrives at the next terminal                    
                yield self.shipment[request[0]].reward_event
                yield self.shipment[request[0]].action_event[action]
            # Determine next state
            current_location, destination, due_time, volume, d_profile, current_time = updated_state
            current_location = self.shipment[request[0]].current_location
            if 'Truck' in current_location:
                current_location = identify_truck_line(current_location)
            d_profile = d_profile_list[-1][0] if d_profile_list else 'no disruption'
            current_time = env.now % (1440*7) // 60
            next_state = current_location, destination, due_time, volume, d_profile, current_time
        # print(f'debug next state: {next_state}')

        # Get reward
        reward = self.shipment[request[0]].reward
        total_reward += reward
        self.shipment[request[0]].reward = 0
        print_event(f'{time_format(env.now)} - {request[0]} got reward: {reward} for action: {action}') #debug

        # Determine next action
        yield env.timeout(1)
        if self.shipment[request[0]].status == "Delivered" or self.shipment[request[0]].status == "Undelivered":
            next_action = 0 # no action after a shipment is delivered
        
        else:
            mode = self.shipment[request[0]].mode[0].name
            if 'Truck' in mode:
                name = identify_truck_line(mode)
                next_action = mode_ID[name]
            else:
                next_action = mode_ID[mode]
        
        print_event(f'{time_format(env.now)} - {request[0]} take next action: {next_action}')
        # self.shipment[request[0]].reward_event = env.event()
        self.shipment[request[0]].action_event[action] = env.event()
        while self.queue:
            yield env.timeout(1)

        # Convert state to vector
        current_location_vector = tuple(self.state_to_vector(updated_state[0], loc_to_index))
        destination_vector = tuple(self.state_to_vector(updated_state[1], dest_to_index))
        profile_vector = tuple(self.state_to_vector(updated_state[4], d_profile_to_index))
        updated_state_vector = current_location_vector, destination_vector, updated_state[2], updated_state[3], profile_vector, updated_state[5]

        current_location_vector = tuple(self.state_to_vector(next_state[0], loc_to_index))
        destination_vector = tuple(self.state_to_vector(next_state[1], dest_to_index))
        profile_vector = tuple(self.state_to_vector(next_state[4], d_profile_to_index))
        next_state_vector = current_location_vector, destination_vector, next_state[2], next_state[3], profile_vector, next_state[5]

        updated_state_tuple = tuple(updated_state_vector)
        next_state_tuple = tuple(next_state_vector)

        self.queue.append([updated_state_tuple, action_taken, reward, next_state_tuple, next_action, request[0]]) #Test debug message "Request[0]"
        self.test_queue.append([updated_state, action_taken, reward, next_state, next_action, request[0]]) #Test debug message "Request[0]"
        self.update_q_table(self.q_table, self.discount_factor, self.alpha)
        for reward_gen in reward_generator[request[0]]:
            if reward_gen[1] == gen_order:
                reward_generator[request[0]].remove(reward_gen)

    def update_q_table(self, Q, discount_factor, alpha):

        # yield self.generate_reward
        state, action, reward, next_state, next_action, request = self.queue[0] 
        
        if state not in Q:
            Q[state] = {}
        if action not in Q[state]:
            Q[state][action] = 0

        if next_state not in Q:
            Q[next_state] = {}
        if next_action not in Q[next_state]:
            Q[next_state][next_action] = 0

        possible_action = {}
        for a in mode_ID.values():
            if get_q_value(Q, next_state, a) != 0:
                possible_action[a] = [Q[next_state][a]]

        if not possible_action:
            best_next_action = 0
        else:
            best_next_action = max(possible_action, key=possible_action.get)

        td_target = reward + discount_factor * get_q_value(Q, next_state, best_next_action)
        print_event(f'{time_format(env.now)} - TEST Q[s,a] before update: {Q[state][action]}')
        print_event(f'{time_format(env.now)} - TEST best next Q[s,a]: {get_q_value(Q, next_state, best_next_action)}')
        td_delta = td_target - get_q_value(Q, state, action)
        Q[state][action] += alpha * td_delta
        print_event(f'{time_format(env.now)} - TEST Q[s,a] after update: {Q[state][action]}')
        self.queue.pop(0)
        self.test_queue.pop(0)
        # self.generate_reward = env.event()

    def state_to_vector(self, state_s, state_to_index):
        vector = [0] * len(state_to_index)
        vector[state_to_index[state_s]] = 1
        return vector

# Define service disruption
class ServiceDisruption():

    def __init__(self, env, mode_schedule, profile):
        self.env = env
        self.disruption_signal = env.event()
        self.mode_schedule = mode_schedule
        self.profile = profile
        self.disruption_sequence = [0]
        self.start_times = []

    def generate_s_disruption(self, profile):

        global truck_name_list
        global disruption_location
        name, type, lbd, ubd, lbc, ubc, possible_location, lambda_rate = profile
        location = 0
        former_disruption = 0
        while True:
            IAT = int(np.random.exponential(scale=1/lambda_rate))
            location = 0
            if IAT != 0:
                start_time = env.now + IAT
                if start_time in self.start_times:
                    IAT += 1 #avoiding conflict for event signal
                    start_time = env.now + IAT
                self.start_times.append(start_time)
                duration = np.random.randint(lbd * 60, ubd * 60)
                capacity_reduction = np.random.uniform(lbc, ubc)
                loc_candidate = np.random.choice(possible_location)
                count = 0
                yield env.timeout(IAT)
                while location in disruption_location: #avoiding the same location
                    if loc_candidate == 'Terminal':
                        location = np.random.choice(node_list)
                    else:
                        mode_candidate_ref = [item for item in mode_list if loc_candidate in item]
                        mode_candidate = []
                        for mode in mode_candidate_ref:
                            # TEST time for disruption
                            operating = self.mode_schedule[mode].departure_time - 210
                            depart = self.mode_schedule[mode].departure_time

                            if 'Truck' in mode:
                                mode_candidate.append(mode)
                            # elif self.mode_schedule[mode].status == 'Operating':
                            elif env.now >= operating and env.now <= depart:
                                mode_candidate.append(mode)
                        if mode_candidate:
                            location = np.random.choice(mode_candidate)
                        else:
                            location = np.random.choice(mode_candidate_ref)
                            location = 0
                            break
                        if 'Truck' in location:
                            if truck_name_list:
                                location = np.random.choice(truck_name_list)
                    count += 1
                    if count > 10:
                        location = 0
                        break
                if location != 0:
                    self.disruption_sequence.append(location)
                    original_capacity = self.mode_schedule[location].capacity if type == 'Capacity reduction' else 0
                    location_new = location
                    disruption_location.append(location_new)
                    if type == 'Capacity reduction':
                        self.mode_schedule[location].capacity = math.ceil((1-capacity_reduction) * self.mode_schedule[location].capacity)
                    d_profile_list.append((name, location))
                    env.process(self.execute_disruption(location, duration, name, type, original_capacity))
                    former_disruption = IAT
                    yield env.timeout(1)
                    # self.start_times.remove(env.now-1)
                    # else:
                    #     yield env.timeout(max(IAT,former_disruption))
            else:
                yield env.timeout(1)

    def produce(self):
        for profile in self.profile:
            env.process(self.generate_s_disruption(profile))
            yield env.timeout(1) # Wait to generate the first disruption
            
    def execute_disruption(self, location, duration, name, type, original_capacity):
        
        global s_disruption_triggers
        # Simulate service disruption
        s_disruption_event[location] = env.event()
        print_event(f"{time_format(env.now)} - Service disruption type {type} at/on {location} starts")
        s_disruption_triggers += 1
        if type == 'Capacity reduction':
             print_event(f"{time_format(env.now)} - {self.mode_schedule[location].name} capacity is reduced to {self.mode_schedule[location].capacity}")
             location_new = location
            #  if 'Truck' in location:
            #      location_new = identify_truck_line(location)
             s_disruption_event[location_new].succeed()
        self.disruption_signal.succeed()

        yield env.timeout(duration)
        if type == 'Capacity reduction':
            self.mode_schedule[location].capacity = original_capacity
            print_event(f"{time_format(env.now)} - {self.mode_schedule[location].name} capacity is restored to {original_capacity}")
        else:
            location_new = location
            # if 'Truck' in location:
            #     location_new = identify_truck_line(location)
            s_disruption_event[location_new].succeed()
        print_event(f"{time_format(env.now)} - Service disruption type {type} at/on {location} ends")
        self.disruption_sequence.remove(location)
        location_new = location
        # if 'Truck' in location:
        #     location_new = identify_truck_line(location)
        disruption_location.remove(location_new)
        d_profile_list.remove((name, location_new))
        # del affected_requests[location_new]

# Define demand disruption
class DemandDisruption():
    def __init__(self, env, shipment, profile):
        self.env = env
        self.shipment = shipment
        self.profile = profile

    def produce(self):
        for profile in self.profile:
            env.process(self.generate_d_disruption(profile))
            yield env.timeout(1) # Wait to generate the first disruption

    def generate_d_disruption(self, profile):
        name, type, lbt, ubt, lbv, ubv, lambda_rate = profile
        # disruption_type = ('Release Time', 'Volume')
        while True:
            #Randomize disruptions
            lambda_rate = lambda_rate
            start_time_d = int(np.random.exponential(scale=1/lambda_rate))
            yield env.timeout(start_time_d)
            disruption = type
            if announced_requests != []:
                disrupted_shipment = np.random.choice(announced_requests)
                d_profile_list.append((name, disrupted_shipment))
                print_event(f"{time_format(env.now)} - Disrupting {disrupted_shipment} with {disruption}")
                if self.shipment[disrupted_shipment].status == "Announced" or self.shipment[disrupted_shipment].status == "Assigned":
                    if disruption == 'Release Time':
                        # Simulate disruption for change in the release time
                        late_release = np.random.randint(lbt*60, ubt*60)
                        self.shipment[disrupted_shipment].release_time += late_release
                        self.shipment[disrupted_shipment].status = "New Release Time"
                        self.shipment[disrupted_shipment].process.interrupt()
                    else:
                        # Simulate disruption for change in the shipment volume
                        volume_multiplier = np.random.uniform(1+lbv, 1+ubv)
                        # Update free capacity for the assigned mode
                        if self.shipment[disrupted_shipment].mode:
                            if not isinstance(self.shipment[disrupted_shipment].mode[0], str):
                                modes = self.shipment[disrupted_shipment].mode
                                for mode in modes:
                                    mode.free_capacity += self.shipment[disrupted_shipment].num_containers
                        self.shipment[disrupted_shipment].num_containers = math.ceil(self.shipment[disrupted_shipment].num_containers * volume_multiplier)
                        self.shipment[disrupted_shipment].status = "New Volume"
                        self.shipment[disrupted_shipment].process.interrupt()
                yield env.timeout(1)
                d_profile_list.remove((name, disrupted_shipment))

# Update cost for undelivered shipments
def update_undelivered_shipments(env, shipment_dict, simulation_duration, penalty):
    global total_storage_time
    global total_storage_cost
    global total_travel_cost
    global total_cost
    global rl_assignment
    global reward_generator

    yield env.timeout(simulation_duration - env.now - 1)
    print_event('\nUPDATE UNDELIVERED SHIPMENTS')
    # print_event("RL Assignment: ", rl_assignment)
    for key, value in shipment_dict.items():
        if value.status != "Delivered":

            if value.status == 'Waiting for arrival':
                storage_time = max(0, env.now - value.release_time)
                shipment_storage_cost = (storage_time / 60) * storage_cost * value.num_containers
                total_storage_time += storage_time
                value.total_storage_cost += shipment_storage_cost
                total_storage_cost += shipment_storage_cost
                total_cost += value.total_storage_cost

                if value.name in rl_assignment:
                    storage_time_rl = max(0, env.now - value.rl_start_time)
                    value.reward += ((storage_time_rl / 60) * storage_cost * value.num_containers) * -1
                    # penalty for undelivered shipments
                    value.reward += penalty * -1
                    value.total_reward += value.reward

            if value.status == 'On board':
                travel_cost1 = value.mode[0].travel_cost1 * (env.now - value.mode[0].actual_departure)/60 *value.num_containers
                travel_cost2 = value.mode[0].travel_cost2 * value.mode[0].distance * value.num_containers
                travel_cost = travel_cost1 + travel_cost2
                value.total_travel_cost += travel_cost
                total_travel_cost += travel_cost
                total_cost += value.total_travel_cost

                if value.name in rl_assignment:
                    value.reward += travel_cost * -1
                    # penalty for undelivered shipments
                    value.reward += penalty * -1
                    value.total_reward += value.reward
            
            value.status = "Undelivered"
            if value.name in rl_assignment:
                value.assigned_to_rl = True
                for process_ID in reward_generator[value.name]:
                    process = process_ID[0]
                    process.interrupt()

                if value.name in rl_assignment:
                    value.reward_event.succeed()
                    if 'Truck' in value.mode[0].name:
                        name = identify_truck_line(value.mode[0].name)
                        value.action_event[name].succeed()
                    else:
                        value.action_event[value.mode[0].name].succeed()

                
# Function to observe the simulation
def observe(env):
    while True:
        obs_time.append(env.now/60)
        obs_storage.append(total_storage_time/60)
        obs_delay.append(total_shipment_delay/60)
        yield env.timeout(60)

# ----- Run the Simulation ----- #

if random_seed:
    print_event(f"Random seed is enabled")
else:
    print_event(f"Random seed is disabled. Seed value: {random_seed_value}")

if start_from_0:
    total_storage_cost_plot = []
    total_travel_cost_plot = []
    total_handling_cost_plot = []
    total_shipment_delay_plot = []
    total_cost_plot = []
    total_reward_plot = []
    total_late_plot = []
    total_number_late_plot = []
    total_rl_triggers = []
    total_assigned_rl = []
    undelivered_requests = []
    last_episode = 0
    total_reassign_plot = []
    total_wait_plot = []
else:
    # total_cost_plot = total_cost_plot_read
    # total_reward_plot = total_reward_plot_read
    # last_episode = len(total_cost_plot_read)
    total_cost_plot = []
    total_reward_plot = []
    total_late_plot = []
    total_number_late_plot = []
    total_rl_triggers = []
    total_assigned_rl = []
    last_episode = 20


x = []
# smoothing = number_of_simulation//100
smoothing = len(total_cost_plot)//100

for simulation in range(number_of_simulation):
    current_episode = last_episode + simulation

    try:
        print_event(f"Simulation number: {simulation + 1} starts")
        env = sim.Environment()
        env.process(clock(env, 1440, simulation))
        env.process(observe(env))

        # Restore possible paths departure time
        possible_paths = possible_paths_ref.copy()

        #Set Global Variables
        announced_requests = []
        active_requests = []
        unassigned_requests = []
        requests_to_replan = []           
        affected_requests = {}
        disruption_location = [0]
        d_profile_list = []
        s_disruption_event = {node: env.event() for node in (node_list + mode_list)}
        total_storage_time = 0
        total_storage_cost = 0
        total_handling_cost = 0
        total_travel_cost = 0
        total_shipment_delay = 0
        total_delay_penalty = 0
        storage_time_list = []
        delivered_shipments = []
        actual_carried_shipments= {mode: 0 for mode in mode_list}
        actual_itinerary = {req_id: [] for req_id in request_ids}
        wait_actions = {req_id: 0 for req_id in request_ids}
        reassign_actions = {req_id: 0 for req_id in request_ids}
        obs_time = []
        obs_storage = []
        obs_delay = []
        truck_waiting_time = 150 # 2.5 hours from the release time to departure time from terminal
        rl_assignment = []
        reward_generator = {}
        total_reward = 0
        total_cost = 0
        rl_number = 0 # for debugging
        rg_order = 0
        om_triggers = 0
        rl_triggers = 0
        s_disruption_triggers = 0
        truck_id = 0
        truck_name_list = []
        assigned_to_rl = []
        nr_late_departure = 0
        total_late_departure = 0
        late_logs = []
        late_dict = {mode: [0, 0] for mode in mode_list}

        # Initiate transport modes
        mode_schedule_dict = {}
        for mode in fixed_list:
            name, schedule, capacity, speed, distance, costs = fixed_schedule_dict[mode]
            mode_schedule_dict[mode] = Mode(env, name, schedule, capacity, speed, distance, costs)
            env.process(mode_schedule_dict[mode].operate())                

        # Initiate shipment requests
        shipment_dict = {}
        for req in request_list:
            shipment = Shipment(env, req)
            shipment_dict[req[0]] = shipment

        rl_module = ReinforcementLearning(env, shipment_dict, mode_schedule_dict, Q, discount_factor=0.9, alpha=0.1)

        planning = MatchingModule(env, mode_schedule_dict, shipment_dict, rl_module)
        env.process(planning.planning())

        # Initiate service disruptions
        s_disruption = ServiceDisruption(env, mode_schedule_dict, s_disruption_profile)
        env.process(s_disruption.produce())

        # Initiate demand disruptions
        d_disruption = DemandDisruption(env, shipment_dict, d_disruption_profile)
        env.process(d_disruption.produce())

        # Initiate affected shipment checker
        env.process(affected_request_detection(env, shipment_dict, s_disruption, planning))

        # Cost updating for undelivered shipments
        env.process(update_undelivered_shipments(env, shipment_dict, simulation_duration, undelivered_penalty))

        if random_seed:
            seed = current_episode
        else:
            seed = random_seed_value
        np.random.seed(seed)

        # Run the simulation until the simulation duration
        env.run(until=simulation_duration)

        total_wait_action = 0
        total_reassign_action = 0
        for rq in request_ids:
            total_wait_action += wait_actions[rq]
            total_reassign_action += reassign_actions[rq]
        
        # Store the total cost and total reward
        total_storage_cost_plot.append(total_storage_cost)
        total_handling_cost_plot.append(total_handling_cost)
        total_travel_cost_plot.append(total_travel_cost)
        total_shipment_delay_plot.append(total_delay_penalty)
        total_cost_plot.append(total_cost)
        total_reward_plot.append(total_reward)
        total_late_plot.append(total_late_departure)
        total_number_late_plot.append(nr_late_departure)
        total_rl_triggers.append(rl_triggers)
        total_assigned_rl.append(len(rl_assignment))
        total_wait_plot.append(total_wait_action)
        total_reassign_plot.append(total_reassign_action)
        total_shipment = len(request_list)
        total_delivered = len(delivered_shipments)
        u_req = total_shipment - total_delivered
        undelivered_requests.append(u_req)
        x.append(simulation+1)
        print(" ")
        print(f"Simulation number {simulation + 1} ends")

        if print_plot == True:
            # Plot the storage time and delay time
            fig, ax1 = plt.subplots()
            ax1.plot(obs_time, obs_storage, label="Total storage time")
            ax1.set_xlabel("Time Step (hours)")
            ax1.set_ylabel("Storage Time (hours)")
            ax1.set_title("Total Storage Time")

            fig, ax2 = plt.subplots()
            ax2.plot(obs_time, obs_delay, label="Total delay time")
            ax2.set_xlabel("Time Step (hours)")
            ax2.set_ylabel("Delay Time (hours)")
            ax2.set_title("Total Delay Time")
            plt.show()

        if print_output == True:
            print("\nService disruptions: ", s_disruption_triggers, " times")
            print("\nOptimization module is triggred: ", om_triggers, " times")
            print("\nReinforcement learning is triggred: ", rl_triggers, " times")
            print("\nTotal late departure: ", total_late_departure, " times")
            print("Total number of late departure: ", nr_late_departure, " times")
            print("Average late departure: ", total_late_departure/nr_late_departure, " minutes")
            print("\nTOTAL COSTS")
            print("----------------------------------------")
            print(f"Total storage cost: {total_storage_cost:.2f} EUR")
            print(f"Total handling cost: {total_handling_cost:.2f} EUR")
            print(f"Total travel cost: {total_travel_cost:.2f} EUR")
            print(f"Total delay penalty: {total_delay_penalty:.2f} EUR")
            print(f"Total cost: {total_cost:.2f} EUR")
            print("----------------------------------------")

            average_storage_time = np.mean(storage_time_list)
            print("\nPERFORMANCE SUMMARY")
            print("----------------------------------------")
            print(f"Average storage time: {average_storage_time/60:.2f} hours/shipment")
            print(f"Total storage time: {total_storage_time/60:.2f} hours")
            print(f"Total delay time: {total_shipment_delay//60:02d} hour(s) {total_shipment_delay%60:02d} minute(s)")
            print("----------------------------------------\n")
            total_shipment = len(request_list)
            total_delivered = len(delivered_shipments)

            print(f"{(total_delivered)} shipment are delivered from total {total_shipment} requests")
            undelivered = set(request['ID']) - set(delivered_shipments)
            print(f"List of undelivered shipment: {undelivered}")
            # Export the episode total cost list
            # with open('total_cost_200.pkl', 'wb') as f:
            #     pickle.dump(total_cost_plot, f)
            #     print("Total cost per episode is exported as 'total_cost_eps.pkl'")
            # with open('total_reward_200.pkl', 'wb') as f:
            #     pickle.dump(total_reward_plot, f)
            #     print("Total reward per episode is exported as 'total_cost_eps.pkl'")
            # with open(f'{q_table_path}', 'wb') as f:
            #     pickle.dump(dict(Q), f)
            # current_episode = last_episode + simulation
            # shipment_logs(shipment_dict, actual_itinerary, actual_carried_shipments, assigned_to_rl, wait_actions, reassign_actions, late_dict, current_episode)
            # if current_episode % 1000 == 0:
            #     print_event(f"Q-table is saved as {q_table_path}")
            #     with open(f'q_table\q_table_200_{current_episode}_eps', 'wb') as f:
            #         pickle.dump(dict(Q), f)

    except Exception as e:
        print(f"Error in simulation number {simulation + 1}: {e}")

# #Plot Total Cost
# plt.figure(figsize=(10, 6))  # Setting the size of the plot
# plt.plot(x, total_cost_plot, marker='o', linestyle='-', color='b')  # Plotting x vs. y
# plt.title('Total Cost per Episode of Simulation')  # Adding a title
# plt.xlabel('Episode')  # Labeling the x-axis
# plt.ylabel('Total Cost')  # Labeling the y-axis
# plt.grid(True)  # Adding a grid for better readability
# plt.show()


# fig = plt.figure(figsize=(10,5))
# costs_smoothed = pd.Series(total_cost_plot).rolling(smoothing, min_periods=smoothing).mean()
# fig = plt.figure(figsize=(10,5))
# plt.plot(costs_smoothed)
# plt.xlabel("Episode")
# plt.ylabel("Episode total cost (Smoothed)")
# plt.title(f"Episode total cost over {seed} episodes)")

# #Plot Q-Learning
# fig = plt.figure(figsize=(10,5))
# rewards_smoothed = pd.Series(total_reward_plot).rolling(smoothing, min_periods=smoothing).mean()
# plt.plot(rewards_smoothed)
# plt.xlabel("Episode")
# plt.ylabel("Episode Reward (Smoothed)")
# plt.title("Episode Reward over Time (Smoothed over window size {})".format(smoothing))

# Saving the Q-table
# with open(f'{q_table_path}', 'wb') as f:
#     pickle.dump(dict(Q), f)

# Create output dataframe
output = pd.DataFrame({'Episode': x,
                       'Total Storage Cost': total_storage_cost_plot,
                       'Total Travel Cost': total_travel_cost_plot,
                       'Total Handling Cost': total_handling_cost_plot,
                       'Total Delay Penalty': total_shipment_delay_plot,
                       'Total Cost': total_cost_plot,
                       'Total Cost': total_cost_plot,
                       'Total Reward': total_reward_plot,
                       'Total Late Departure': total_late_plot,
                       'Number of Late Departure': total_number_late_plot,
                       'RL Triggers': total_rl_triggers,
                       'Shipment to RL': total_assigned_rl, 
                       'Undelivered Requests': undelivered_requests,
                       'Wait Actions': total_wait_plot, 
                       'Reassign Actions': total_reassign_plot
                       })
# output.to_csv(f'{output_path}', index=False)

# # Export the episode total cost list
# with open(f'{experiment_path}', 'wb') as f:
#     pickle.dump(total_cost_plot, f)

Random seed is enabled
Simulation number: 1 starts
 
current day: 1, simulation: 1
00:00 - Request1 with 7 containers requests transport from Delta to Neuss
00:00 - Request10 with 14 containers requests transport from Delta to Dortmund
00:00 - Request11 with 15 containers requests transport from Delta to Duisburg
00:00 - Request12 with 9 containers requests transport from Delta to Dortmund
00:00 - Request13 with 6 containers requests transport from Euromax to Neuss
00:00 - Request14 with 27 containers requests transport from Euromax to Duisburg
00:00 - Request15 with 15 containers requests transport from Delta to Venlo
00:00 - Request16 with 22 containers requests transport from HOME to Duisburg
00:00 - Request17 with 11 containers requests transport from Delta to Dortmund
00:00 - Request18 with 21 containers requests transport from Delta to Venlo
00:00 - Request19 with 7 containers requests transport from HOME to Nuremberg
00:00 - Request2 with 29 containers requests transport from De